In [0]:
import mlflow.deployments

## What this notebook does
1. Create secret scope for OpenAI API key
2. Create external model endpoint pointing to OpenAI o3-mini
3. Create tables in Unity Catalog to store document metadata and text

## Create secret scope for OpenAI and Mistral API key


In a terminal run the following to create a scope, add your API key, and confirm scope creation
<br/> <br/>
```
databricks secrets create-scope openai
databricks secrets put-secret openai apikey
databricks secrets create-scope mistral
databricks secrets put-secret mistral apikey
databricks secrets list-scopes
```

##Create external model endpoint pointing to OpenAI o3-mini

In [0]:
client = mlflow.deployments.get_deploy_client("databricks")

endpoint = client.create_endpoint(
    name="openai-completion-endpoint",
    config={
        "served_entities": [
            {
                "name": "completions",
                "external_model": {
                    "name": "o3-mini",
                    "provider": "openai",
                    "task": "llm/v1/chat",
                    "openai_config": {
                        "openai_api_key": "{{secrets/openai/apikey}}",
                    },
                },
            }
        ],
    },
)

## Create tables in Unity Catalog to store document metadata and text

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_examples.financial_rag.pdf_metadata (
    fileNumber BIGINT PRIMARY KEY,
    companyName STRING,
    tradingSymbol STRING,
    fiscalYearEndDate STRING,
    documentHash STRING
) TBLPROPERTIES (delta.enableChangeDataFeed = true);

CREATE TABLE IF NOT EXISTS databricks_examples.financial_rag.pdf_markdown_text (
    id BIGINT GENERATED ALWAYS AS IDENTITY,
    fileNumber BIGINT,
    markdownText STRING,
    pageNumber INT,
    FOREIGN KEY (fileNumber) REFERENCES databricks_examples.financial_rag.pdf_metadata(fileNumber)
) TBLPROPERTIES (delta.enableChangeDataFeed = true);